```python
# TODO: run it once finished to create the package requirements file
pip freeze > requirements.txt
# conda list -e > requirements.txt
```

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from scipy.stats import uniform, randint

import wget
import os.path

Let's load our source file:

In [2]:
if not os.path.isfile('data/listings.csv.gz'):
    wget.download('http://data.insideairbnb.com/australia/vic/melbourne/2023-09-04/data/listings.csv.gz', out = 'data/listings.csv.gz')

In [3]:
df_listings = pd.read_csv('data/listings.csv.gz')

Drop unnecessary columns identified during EDA:

In [4]:
df_listings = df_listings.drop(['neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated', 'license', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'host_url', 
                                'host_since', 'host_about', 'host_location', 'calendar_last_scraped', 'description', 'neighborhood_overview', 'picture_url', 'host_thumbnail_url', 'host_picture_url',
                                'name', 'host_name', 'host_verifications', 'host_listings_count', 'host_total_listings_count', 'host_neighbourhood', 'host_has_profile_pic', 'property_type', 'amenities', 'minimum_minimum_nights',
                                'minimum_nights_avg_ntm', 'maximum_minimum_nights', 'maximum_nights_avg_ntm', 'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 
                                'number_of_reviews_ltm', 'number_of_reviews_l30d', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'], 
                                axis=1)

Transformations for both, ensemble model and linear regressor:

In [5]:
df_listings['bathrooms_type'] = df_listings['bathrooms_text'].str.lower()\
                                                            .str.extract('(shared|private)')

df_listings['bathrooms_qty'] = df_listings['bathrooms_text'].str.lower()\
                                                            .str.replace('half', '0.5')\
                                                            .str.extract(r'([0-9]+\.?[0-9]*)')

In [6]:
# Impute Entire home/apt type of property to private bathroom
df_listings.loc[((df_listings['bathrooms_type'].isna()) & (df_listings['room_type'] == 'Entire home/apt')), 'bathrooms_type'] = 'private'

In [7]:
# Impute remaining missing with the mode 
df_listings.loc[df_listings['bathrooms_type'].isna(), 'bathrooms_type'] = df_listings['bathrooms_type'].mode().values[0]

In [8]:
df_listings['host_response_rate'] = df_listings['host_response_rate'].str.replace('%', '')
df_listings['host_acceptance_rate'] = df_listings['host_acceptance_rate'].str.replace('%', '')

In [9]:
df_listings['price'] = df_listings['price'].str.replace(r'[$,]', '', regex=True)

In [10]:
# Drop unnecessary columns and features. Arguably, the don't offer any value for the purpose of this project
# For example, neighbourhood was dropped because we'll use neighbourhood_cleansed instead, which doesn't have any missing values and a lower cardinality
df_listings = df_listings.drop(columns=['id', 'first_review', 'last_review', 'host_id', 
                              'calculated_host_listings_count', 'neighbourhood', 'bathrooms_text'])

Transformations for LR model:

In [11]:
df_listings_lr = df_listings.copy()

In [12]:
df_listings_lr = df_listings_lr.astype({
    'instant_bookable': 'bool',
    'host_response_time': 'category', 
    'host_acceptance_rate': 'category', 
    'host_is_superhost': 'category', 
    'room_type': 'category',
    'host_response_rate': 'float',
    'host_acceptance_rate': 'float',
    'price': 'float',
    'bathrooms_qty': 'float'
})

df_listings_lr['host_response_rate'] = df_listings_lr['host_response_rate'] / 100
df_listings_lr['host_acceptance_rate'] = df_listings_lr['host_acceptance_rate'] / 100
df_listings_lr['price'] = df_listings_lr['price'].round()\
                                                .astype('int')

In [13]:
# Show all columns (instead of cascading columns in the middle)
pd.set_option("display.max_columns", None)

A quick sneak peak to the data:

In [14]:
df_listings.head()

,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_maximum_nights,maximum_maximum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,bathrooms_type,bathrooms_qty
0,within a few hours,100,95,f,t,Moreland,-37.76606,144.97951,Private room,2,NaN,1.0,49.00,5,14,14,14,173,4.49,4.65,3.98,4.72,4.69,4.66,4.61,f,1.33,shared,1
1,NaN,NaN,NaN,f,t,Port Phillip,-37.85999,144.97662,Entire home/apt,2,1.0,1.0,95.00,3,14,14,14,42,4.68,4.78,4.71,4.83,4.83,4.78,4.66,f,0.26,private,1
2,within an hour,100,91,t,t,Casey,-38.05723,145.33982,Entire home/apt,5,3.0,3.0,116.00,1,14,14,14,228,4.86,4.92,4.98,4.91,4.94,4.90,4.88,f,1.47,private,1
3,NaN,NaN,NaN,f,t,Darebin,-37.69761,145.00066,Private room,2,NaN,1.0,40.00,7,365,1125,1125,159,4.71,4.68,4.65,4.89,4.83,4.39,4.69,f,1.02,shared,1
4,within an hour,100,99,t,t,Monash,-37.89983,145.11579,Entire home/apt,2,1.0,1.0,117.00,2,1125,1125,1125,248,4.87,4.91,4.93,4.94,4.93,4.79,4.86,f,1.60,private,1


Checking dataset types:

TODO:
- ~~Decide strategy for category attributes~~
- ~~Explore which attributes would need normalization~~
- ~~Decide regression algorithm~~
- How to deal with location?

Let's separate target attribute and the rest of dataset and split the dataset on train and test sets and set the strategy for preprocessing:

In [15]:
# Define the target variable
target_variable = 'price'

# Define categorical and numerical features
categorical_features = ['host_response_time', 'host_is_superhost', 'host_identity_verified',
                        'neighbourhood_cleansed', 'room_type', 'bathrooms_type', 'instant_bookable']

numerical_features = ['host_response_rate', 'host_acceptance_rate', 'latitude', 'longitude',
                      'accommodates', 'bedrooms', 'beds', 'minimum_nights', 'maximum_nights',
                      'minimum_maximum_nights', 'maximum_maximum_nights', 'number_of_reviews',
                      'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
                      'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'bathrooms_qty',
                      'review_scores_value', 'reviews_per_month']

# Create preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

Now, let's use a simple linear regressor for our baseline model:

In [16]:
# Handle missing values with imputation bfill or ffill
def handle_missing_values(df):
    df.ffill(inplace=True)
    df.bfill(inplace=True)
    return df


Let's split the dataset into train and test sets, and set apart the target variable:

In [17]:
y = df_listings_lr[target_variable]
X = df_listings_lr.drop([target_variable], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, 
                                              y,
                                              test_size=0.2,
                                              random_state=99
                                              )

In [18]:
# Instantiate the linear regression model
linear_model = LinearRegression()

# Create pipeline with preprocessor and model
pipeline_linear = Pipeline(steps=[('preprocessor', preprocessor),
                           ('estimator', linear_model)])

pipeline_linear.fit(handle_missing_values(X_train), y_train)

# Predict the prices
y_pred_train = pipeline_linear.predict(handle_missing_values(X_train))
y_pred_test = pipeline_linear.predict(handle_missing_values(X_test))

# evaluate the model with cross-validation and mean absolute error on the test set
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)

print(f"Train MAE: {train_mae}")
print(f"Test MAE: {test_mae}")

Train MAE: 155.76535026940235
Test MAE: 154.94405520473904


In [19]:
# let's plot the predictions vs. the actual values
fig = px.scatter(x=y_test, 
                y=y_pred_test      
                )
fig.update_layout(
    title='Linear model - predictions vs. actual values',
    xaxis_title="Actuals",
    yaxis_title="Predictions"
)

In [20]:
# let's plot the residuals
residuals = y_test - y_pred_test
px.histogram(x=residuals)

In [21]:
# let's create a dataframe with the actual, predicted values and the residuals
df_pred = pd.DataFrame({'actual': y_test, 'predicted': y_pred_test, 'residuals': y_test - y_pred_test})

In [22]:
df_pred.sort_values(by='residuals', ascending=False).head(10)

,actual,predicted,residuals
15498,85000,715.144531,84284.855469
3492,13379,88.832031,13290.167969
13542,9180,426.144531,8753.855469
5136,8000,152.441406,7847.558594
3934,4262,136.281250,4125.718750
10783,3988,235.804688,3752.195312
7967,4000,536.957031,3463.042969
10781,3584,257.457031,3326.542969
8772,3000,33.269531,2966.730469
6097,3000,314.894531,2685.105469


We would pass any tweaks or tuning for now as this is a baseline model.

Now let's setup the dateset for train the boosting model and see if we can get a better performance than our baseline model:

In [43]:
y = df_listings[target_variable]
X = df_listings.drop([target_variable], axis=1)

In [44]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_validate, RandomizedSearchCV

histogram_gradient_boosting = HistGradientBoostingRegressor(
    max_iter=300, 
    random_state=0,
    early_stopping=True
)

# Define parameter grid for RandomizedSearchCV
param_dist = {
    'learning_rate': uniform(0.01, 0.1),
    'max_leaf_nodes': randint(20, 200)
}

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=histogram_gradient_boosting, 
                                   param_distributions=param_dist, 
                                   scoring='neg_mean_absolute_error',
                                   cv=5, 
                                   n_iter=10, # Tested with 30, no real difference 
                                   random_state=42
                                   )

# Create pipeline with preprocessor and model
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('estimator', random_search)])


In [45]:
# Train the model
cv_results_hgbdt = cross_validate(
    pipeline,
    X,
    y,
    scoring="neg_mean_absolute_error",
    n_jobs=2,
)                            

In [47]:
print("Histogram Gradient Boosting Regressor Tree")
print(
    "Mean absolute error via cross-validation: "
    f"{-cv_results_hgbdt['test_score'].mean():.3f} ± "
    f"{cv_results_hgbdt['test_score'].std():.3f} $"
)
print(f"Average fit time: {cv_results_hgbdt['fit_time'].mean():.3f} seconds")
print(
    f"Average score time: {cv_results_hgbdt['score_time'].mean():.3f} seconds"
)

Histogram Gradient Boosting Regressor Tree
Mean absolute error via cross-validation: 145.394 ± 15.281 $
Average fit time: 161.128 seconds
Average score time: 0.058 seconds


Let's plot the target variable to check the MAE in this context:

In [53]:
# Price histogram. max $500 is a sensible value considering the current price distribution (long tail after $500)
price = df_listings['price'].astype('float')
fig = px.histogram(x=price[price < 500])

fig.update_layout(
    title='Target variable distribution',
    xaxis_title="Price"
)

For small price values, the current model might be problematic considering the variance